In [ ]:
import re
import pickle
import warnings
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from sklearn.impute import SimpleImputer
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))

In [ ]:
df = pd.read_csv('IPL 1st Innings.csv')
df.head()

In [ ]:
df.columns

In [4]:
df = df[['date', 'venue', 'toss_decision', 'batting_team', 'bowling_team', 'overs', 'ball_no', 'runs', 'wickets', 'runs_last_5_overs', 
         'wickets_last_5_overs', 'final_score']]

In [5]:
df.head()

,date,venue,toss_decision,batting_team,bowling_team,overs,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,final_score
0,18-04-2008,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,1,0,1,0,222
1,18-04-2008,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,1,0,1,0,222
2,18-04-2008,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,2,0,2,0,222
3,18-04-2008,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,2,0,2,0,222
4,18-04-2008,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,2,0,2,0,222


In [6]:
df.dtypes

date                    object
venue                   object
toss_decision           object
batting_team            object
bowling_team            object
overs                    int64
ball_no                  int64
runs                     int64
wickets                  int64
runs_last_5_overs        int64
wickets_last_5_overs     int64
final_score              int64
dtype: object

In [8]:
df.head()

,venue,toss_decision,batting_team,bowling_team,overs,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,final_score,year
0,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,1,0,1,0,222.0,2008
1,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,1,0,1,0,222.0,2008
2,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,2,0,2,0,222.0,2008
3,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,2,0,2,0,222.0,2008
4,M Chinnaswamy Stadium,field,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,2,0,2,0,222.0,2008


In [9]:
df.columns

Index(['venue', 'toss_decision', 'batting_team', 'bowling_team', 'overs',
       'ball_no', 'runs', 'wickets', 'runs_last_5_overs',
       'wickets_last_5_overs', 'final_score', 'year'],
      dtype='object')

In [11]:
dummy = ['venue', 'toss_decision', 'batting_team', 'bowling_team']
df_dummy = pd.get_dummies(df[dummy], prefix_sep = '_', drop_first = False)
df = df.drop(dummy, axis = 1)
df = pd.concat([df, df_dummy], axis = 1)

In [12]:
df.head()

,overs,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,final_score,year,"venue_Arun Jaitley Stadium, Delhi",venue_Barabati Stadium,venue_Brabourne Stadium,"venue_Brabourne Stadium, Mumbai",venue_Buffalo Park,venue_De Beers Diamond Oval,venue_Dr DY Patil Sports Academy,venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium,venue_Dubai International Cricket Stadium,venue_Eden Gardens,venue_Green Park,venue_Himachal Pradesh Cricket Association Stadium,venue_Holkar Cricket Stadium,venue_JSCA International Stadium Complex,venue_Kingsmead,venue_M Chinnaswamy Stadium,venue_MA Chidambaram Stadium,"venue_MA Chidambaram Stadium, Chepauk","venue_Narendra Modi Stadium, Ahmedabad",venue_Nehru Stadium,venue_New Wanderers Stadium,venue_Newlands,venue_OUTsurance Oval,venue_Punjab Cricket Association IS Bindra Stadium,"venue_Punjab Cricket Association Stadium, Mohali",venue_Rajiv Gandhi International Stadium,"venue_Rajiv Gandhi International Stadium, Uppal",venue_Saurashtra Cricket Association Stadium,venue_Sawai Mansingh Stadium,venue_Shaheed Veer Narayan Singh International Stadium,venue_Sharjah Cricket Stadium,venue_Sheikh Zayed Stadium,venue_St George's Park,venue_Subrata Roy Sahara Stadium,venue_SuperSport Park,"venue_Vidarbha Cricket Association Stadium, Jamtha",venue_Wankhede Stadium,"venue_Zayed Cricket Stadium, Abu Dhabi",toss_decision_bat,toss_decision_field,batting_team_Chennai Super Kings,batting_team_Deccan Chargers,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kochi Tuskers Kerala,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Punjab Kings,batting_team_Rajasthan Royals,batting_team_Rising Pune Supergiant,batting_team_Royal Challengers Bangalore,batting_team_Sunrisers Hyderabad,bowling_team_Chennai Super Kings,bowling_team_Deccan Chargers,bowling_team_Delhi Capitals,bowling_team_Gujarat Lions,bowling_team_Kochi Tuskers Kerala,bowling_team_Kolkata Knight Riders,bowling_team_Mumbai Indians,bowling_team_Pune Warriors,bowling_team_Punjab Kings,bowling_team_Rajasthan Royals,bowling_team_Rising Pune Supergiant,bowling_team_Royal Challengers Bangalore,bowling_team_Sunrisers Hyderabad
0,0,1,1,0,1,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,2,1,0,1,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,3,2,0,2,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,3,2,0,2,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,4,2,0,2,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [13]:
df.shape

(107664, 74)

In [14]:
df.columns

Index(['overs', 'ball_no', 'runs', 'wickets', 'runs_last_5_overs',
       'wickets_last_5_overs', 'final_score', 'year',
       'venue_Arun Jaitley Stadium, Delhi', 'venue_Barabati Stadium',
       'venue_Brabourne Stadium', 'venue_Brabourne Stadium, Mumbai',
       'venue_Buffalo Park', 'venue_De Beers Diamond Oval',
       'venue_Dr DY Patil Sports Academy',
       'venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'venue_Dubai International Cricket Stadium', 'venue_Eden Gardens',
       'venue_Green Park',
       'venue_Himachal Pradesh Cricket Association Stadium',
       'venue_Holkar Cricket Stadium',
       'venue_JSCA International Stadium Complex', 'venue_Kingsmead',
       'venue_M Chinnaswamy Stadium', 'venue_MA Chidambaram Stadium',
       'venue_MA Chidambaram Stadium, Chepauk',
       'venue_Narendra Modi Stadium, Ahmedabad', 'venue_Nehru Stadium',
       'venue_New Wanderers Stadium', 'venue_Newlands',
       'venue_OUTsurance Oval',
       'venue_Punjab

In [15]:
columns = []
for i in df.columns:
    i = re.sub('\s', '_', i)
    i = re.sub('\.', '', i)
    i = re.sub('\,', '', i)
    i = re.sub('\-', '_', i)
    columns.append(i)
df.columns = columns

In [16]:
df.columns

Index(['overs', 'ball_no', 'runs', 'wickets', 'runs_last_5_overs',
       'wickets_last_5_overs', 'final_score', 'year',
       'venue_Arun_Jaitley_Stadium_Delhi', 'venue_Barabati_Stadium',
       'venue_Brabourne_Stadium', 'venue_Brabourne_Stadium_Mumbai',
       'venue_Buffalo_Park', 'venue_De_Beers_Diamond_Oval',
       'venue_Dr_DY_Patil_Sports_Academy',
       'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium',
       'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens',
       'venue_Green_Park',
       'venue_Himachal_Pradesh_Cricket_Association_Stadium',
       'venue_Holkar_Cricket_Stadium',
       'venue_JSCA_International_Stadium_Complex', 'venue_Kingsmead',
       'venue_M_Chinnaswamy_Stadium', 'venue_MA_Chidambaram_Stadium',
       'venue_MA_Chidambaram_Stadium_Chepauk',
       'venue_Narendra_Modi_Stadium_Ahmedabad', 'venue_Nehru_Stadium',
       'venue_New_Wanderers_Stadium', 'venue_Newlands',
       'venue_OUTsurance_Oval',
       'venue_Punjab_Cricke

In [17]:
df.head()

,overs,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,final_score,year,venue_Arun_Jaitley_Stadium_Delhi,venue_Barabati_Stadium,venue_Brabourne_Stadium,venue_Brabourne_Stadium_Mumbai,venue_Buffalo_Park,venue_De_Beers_Diamond_Oval,venue_Dr_DY_Patil_Sports_Academy,venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium,venue_Dubai_International_Cricket_Stadium,venue_Eden_Gardens,venue_Green_Park,venue_Himachal_Pradesh_Cricket_Association_Stadium,venue_Holkar_Cricket_Stadium,venue_JSCA_International_Stadium_Complex,venue_Kingsmead,venue_M_Chinnaswamy_Stadium,venue_MA_Chidambaram_Stadium,venue_MA_Chidambaram_Stadium_Chepauk,venue_Narendra_Modi_Stadium_Ahmedabad,venue_Nehru_Stadium,venue_New_Wanderers_Stadium,venue_Newlands,venue_OUTsurance_Oval,venue_Punjab_Cricket_Association_IS_Bindra_Stadium,venue_Punjab_Cricket_Association_Stadium_Mohali,venue_Rajiv_Gandhi_International_Stadium,venue_Rajiv_Gandhi_International_Stadium_Uppal,venue_Saurashtra_Cricket_Association_Stadium,venue_Sawai_Mansingh_Stadium,venue_Shaheed_Veer_Narayan_Singh_International_Stadium,venue_Sharjah_Cricket_Stadium,venue_Sheikh_Zayed_Stadium,venue_St_George's_Park,venue_Subrata_Roy_Sahara_Stadium,venue_SuperSport_Park,venue_Vidarbha_Cricket_Association_Stadium_Jamtha,venue_Wankhede_Stadium,venue_Zayed_Cricket_Stadium_Abu_Dhabi,toss_decision_bat,toss_decision_field,batting_team_Chennai_Super_Kings,batting_team_Deccan_Chargers,batting_team_Delhi_Capitals,batting_team_Gujarat_Lions,batting_team_Kochi_Tuskers_Kerala,batting_team_Kolkata_Knight_Riders,batting_team_Mumbai_Indians,batting_team_Pune_Warriors,batting_team_Punjab_Kings,batting_team_Rajasthan_Royals,batting_team_Rising_Pune_Supergiant,batting_team_Royal_Challengers_Bangalore,batting_team_Sunrisers_Hyderabad,bowling_team_Chennai_Super_Kings,bowling_team_Deccan_Chargers,bowling_team_Delhi_Capitals,bowling_team_Gujarat_Lions,bowling_team_Kochi_Tuskers_Kerala,bowling_team_Kolkata_Knight_Riders,bowling_team_Mumbai_Indians,bowling_team_Pune_Warriors,bowling_team_Punjab_Kings,bowling_team_Rajasthan_Royals,bowling_team_Rising_Pune_Supergiant,bowling_team_Royal_Challengers_Bangalore,bowling_team_Sunrisers_Hyderabad
0,0,1,1,0,1,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,2,1,0,1,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,3,2,0,2,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,3,2,0,2,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,4,2,0,2,0,222.0,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [18]:
df = df[['year', 'batting_team_Chennai_Super_Kings', 'batting_team_Deccan_Chargers', 'batting_team_Delhi_Capitals', 
         'batting_team_Gujarat_Lions', 'batting_team_Kochi_Tuskers_Kerala', 'batting_team_Kolkata_Knight_Riders', 
         'batting_team_Mumbai_Indians', 'batting_team_Pune_Warriors', 'batting_team_Punjab_Kings', 'batting_team_Rajasthan_Royals', 
         'batting_team_Rising_Pune_Supergiant', 'batting_team_Royal_Challengers_Bangalore', 'batting_team_Sunrisers_Hyderabad', 
         'bowling_team_Chennai_Super_Kings', 'bowling_team_Deccan_Chargers', 'bowling_team_Delhi_Capitals', 'bowling_team_Gujarat_Lions', 
         'bowling_team_Kochi_Tuskers_Kerala', 'bowling_team_Kolkata_Knight_Riders', 'bowling_team_Mumbai_Indians', 
         'bowling_team_Pune_Warriors', 'bowling_team_Punjab_Kings', 'bowling_team_Rajasthan_Royals', 'bowling_team_Rising_Pune_Supergiant', 
         'bowling_team_Royal_Challengers_Bangalore', 'bowling_team_Sunrisers_Hyderabad', 'toss_decision_bat', 'toss_decision_field', 
         'venue_Arun_Jaitley_Stadium_Delhi', 'venue_Barabati_Stadium', 'venue_Brabourne_Stadium', 'venue_Brabourne_Stadium_Mumbai', 
         'venue_Buffalo_Park', 'venue_De_Beers_Diamond_Oval', 'venue_Dr_DY_Patil_Sports_Academy', 
         'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium', 'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens', 
         'venue_Green_Park', 'venue_Himachal_Pradesh_Cricket_Association_Stadium', 'venue_Holkar_Cricket_Stadium', 
         'venue_JSCA_International_Stadium_Complex', 'venue_Kingsmead', 'venue_M_Chinnaswamy_Stadium', 'venue_MA_Chidambaram_Stadium', 
         'venue_MA_Chidambaram_Stadium_Chepauk', 'venue_Narendra_Modi_Stadium_Ahmedabad', 'venue_Nehru_Stadium', 'venue_New_Wanderers_Stadium', 
         'venue_Newlands', 'venue_OUTsurance_Oval', 'venue_Punjab_Cricket_Association_IS_Bindra_Stadium', 
         'venue_Punjab_Cricket_Association_Stadium_Mohali', 'venue_Rajiv_Gandhi_International_Stadium', 
         'venue_Rajiv_Gandhi_International_Stadium_Uppal', 'venue_Saurashtra_Cricket_Association_Stadium', 'venue_Sawai_Mansingh_Stadium', 
         'venue_Shaheed_Veer_Narayan_Singh_International_Stadium', 'venue_Sharjah_Cricket_Stadium', 'venue_Sheikh_Zayed_Stadium', 
         'venue_St_George\'s_Park', 'venue_Subrata_Roy_Sahara_Stadium', 'venue_SuperSport_Park', 
         'venue_Vidarbha_Cricket_Association_Stadium_Jamtha', 'venue_Wankhede_Stadium', 'venue_Zayed_Cricket_Stadium_Abu_Dhabi', 'overs', 
         'ball_no', 'runs', 'wickets', 'runs_last_5_overs', 'wickets_last_5_overs', 'final_score']]

In [19]:
train = df[df['year'] <= 2018]
test = df[df['year'] > 2018]

In [20]:
X_train = pd.DataFrame(train.drop(['year', 'final_score'], axis = 1), columns = train.drop(['year', 'final_score'], axis = 1).columns)
X_test = pd.DataFrame(test.drop(['year', 'final_score'], axis = 1), columns = test.drop(['year', 'final_score'], axis = 1).columns)
y_train = train['final_score']
y_test = test['final_score']

In [21]:
X_train.columns

Index(['batting_team_Chennai_Super_Kings', 'batting_team_Deccan_Chargers',
       'batting_team_Delhi_Capitals', 'batting_team_Gujarat_Lions',
       'batting_team_Kochi_Tuskers_Kerala',
       'batting_team_Kolkata_Knight_Riders', 'batting_team_Mumbai_Indians',
       'batting_team_Pune_Warriors', 'batting_team_Punjab_Kings',
       'batting_team_Rajasthan_Royals', 'batting_team_Rising_Pune_Supergiant',
       'batting_team_Royal_Challengers_Bangalore',
       'batting_team_Sunrisers_Hyderabad', 'bowling_team_Chennai_Super_Kings',
       'bowling_team_Deccan_Chargers', 'bowling_team_Delhi_Capitals',
       'bowling_team_Gujarat_Lions', 'bowling_team_Kochi_Tuskers_Kerala',
       'bowling_team_Kolkata_Knight_Riders', 'bowling_team_Mumbai_Indians',
       'bowling_team_Pune_Warriors', 'bowling_team_Punjab_Kings',
       'bowling_team_Rajasthan_Royals', 'bowling_team_Rising_Pune_Supergiant',
       'bowling_team_Royal_Challengers_Bangalore',
       'bowling_team_Sunrisers_Hyderabad', 'to

In [22]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((85406, 72), (22258, 72), (85406,), (22258,))

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [25]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
print('Linear Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_lr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_lr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_lr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_lr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_lr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test))
error_lr = (np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test) / 2)

Linear Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5302523164475207
Test r2_score:  -24756170893681.152
----------------------------------------------------------------------------------------------------
Train RMSE:  20.328862400277014
Test RMSE:  149930897.57751572
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09687881680154775
Test MAPE:  198705.47922340504


In [26]:
la = Lasso()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
la_rscv = RandomizedSearchCV(estimator = la, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
                             verbose = 2, n_jobs = 1)
la_rscv.fit(X_train, y_train)
la_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END ........................................alpha=0.916; total time=   0.9s
[CV] END ........................................alpha=0.916; total time=   0.7s
[CV] END ........................................alpha=0.916; total time=   0.7s
[CV] END ........................................alpha=0.916; total time=   0.4s
[CV] END ........................................alpha=0.916; total time=   0.4s
[CV] END ........................................alpha=0.916; total time=   0.4s
[CV] END ........................................alpha=0.916; total time=   0.4s
[CV] END ........................................alpha=0.916; total time=   0.5s
[CV] END ........................................alpha=0.916; total time=   0.4s
[CV] END ........................................alpha=0.916; total time=   0.4s
[CV] END ........................................alpha=0.423; total time=   0.7s
[CV] END .....................................

{'alpha': 0.219}

In [27]:
y_train_pred_la = la_rscv.predict(X_train)
y_test_pred_la = la_rscv.predict(X_test)
print('Lasso Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_la))
print('Test r2_score: ', r2_score(y_test, y_test_pred_la))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_la)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_la)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_la) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test))
error_la = (np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test) / 2)

Lasso Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5038642720665953
Test r2_score:  0.48674679914588515
----------------------------------------------------------------------------------------------------
Train RMSE:  20.89204747577919
Test RMSE:  21.588140615165916
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09873071630730808
Test MAPE:  0.10197384354419099


In [28]:
ri = Ridge()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
ri_rscv = RandomizedSearchCV(estimator = ri, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
                             verbose = 2, n_jobs = 1)
ri_rscv.fit(X_train, y_train)
ri_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.667; total time=   0.1s
[CV] END ........................................alpha=0.565; total time=   0.1s
[CV] END .....................................

{'alpha': 0.859}

In [29]:
y_train_pred_ri = ri_rscv.predict(X_train)
y_test_pred_ri = ri_rscv.predict(X_test)
print('Ridge Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_ri))
print('Test r2_score: ', r2_score(y_test, y_test_pred_ri))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_ri)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_ri)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_ri) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test))
error_ri = (np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test) / 2)

Ridge Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5302827340183913
Test r2_score:  0.36760594103517197
----------------------------------------------------------------------------------------------------
Train RMSE:  20.328204212227522
Test RMSE:  23.963116304611756
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09685146995500837
Test MAPE:  0.11282303729016258


In [23]:
# dtr = DecisionTreeRegressor()
# possible_parameter_values = {'criterion' : ['mse', 'friedman_mse', 'mae'],
#                             'splitter' : ['best', 'random'],
#                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
#                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
#                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
# dtr_rscv = RandomizedSearchCV(estimator = dtr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
#                               verbose = 2, n_jobs = 1)
# dtr_rscv.fit(X_train, y_train)
# dtr_rscv.best_params_

In [24]:
#y_train_pred_dtr = dtr_rscv.predict(X_train)
#y_test_pred_dtr = dtr_rscv.predict(X_test)
#print('Decision Tree Regressor:')
#print('-' * 100)
#print('Train r2_score: ', r2_score(y_train, y_train_pred_dtr))
#print('Test r2_score: ', r2_score(y_test, y_test_pred_dtr))
#print('-' * 100)
#print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_dtr)))
#print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_dtr)))
#print('-' * 100)
#print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_dtr) / y_train))
#print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_dtr) / y_test))
#error_dtr = np.mean(mean_absolute_error(y_test, y_test_pred_dtr) / y_test)

In [26]:
# rfr = RandomForestRegressor()
# possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
#                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
#                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
#                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
# rfr_rscv = RandomizedSearchCV(estimator = rfr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
#                               verbose = 2, n_jobs = 1)
# rfr_rscv.fit(X_train, y_train)
# rfr_rscv.best_params_

In [27]:
#y_train_pred_rfr = rfr_rscv.predict(X_train)
#y_test_pred_rfr = rfr_rscv.predict(X_test)
#print('Random Forest Regressor:')
#print('-' * 100)
#print('Train r2_score: ', r2_score(y_train, y_train_pred_rfr))
#print('Test r2_score: ', r2_score(y_test, y_test_pred_rfr))
#print('-' * 100)
#print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_rfr)))
#print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_rfr)))
#print('-' * 100)
#print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_rfr) / y_train))
#print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_rfr) / y_test))
#error_rfr = np.mean(mean_absolute_error(y_test, y_test_pred_rfr) / y_test)

In [29]:
# abr = AdaBoostRegressor()
# possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
#                              'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
# abr_rscv = RandomizedSearchCV(estimator = abr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
#                               verbose = 2, n_jobs = 1)
# abr_rscv.fit(X_train, y_train)
# abr_rscv.best_params_

In [30]:
# y_train_pred_abr = abr_rscv.predict(X_train)
# y_test_pred_abr = abr_rscv.predict(X_test)
# print('Ada Boost Regressor:')
# print('-' * 100)
# print('Train r2_score: ', r2_score(y_train, y_train_pred_abr))
# print('Test r2_score: ', r2_score(y_test, y_test_pred_abr))
# print('-' * 100)
# print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_abr)))
# print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_abr)))
# print('-' * 100)
# print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_abr) / y_train))
# print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test))
# error_abr = (np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test) / 2)

In [31]:
knr = KNeighborsRegressor()
possible_parameter_values = {'n_neighbors' : [int(x) for x in np.arange(start = 1, stop = 11, step = 1)],
                             'weights' : ['uniform', 'distance']}
knr_rscv = RandomizedSearchCV(estimator = knr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
                              verbose = 2, n_jobs = 1)
knr_rscv.fit(X_train, y_train)
knr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END ...................n_neighbors=10, weights=distance; total time=  10.4s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.6s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.4s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.5s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.5s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.5s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.4s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.5s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.5s
[CV] END ...................n_neighbors=10, weights=distance; total time=   8.5s
[CV] END ....................n_neighbors=7, weights=distance; total time=   8.3s
[CV] END ....................n_neighbors=7, we

{'weights': 'uniform', 'n_neighbors': 10}

In [25]:
# knr_rscv = KNeighborsRegressor(weights = 'uniform', n_neighbors = 10)
# knr_rscv.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=10)

In [27]:
# y_train_pred_knr = knr_rscv.predict(X_train)
# y_test_pred_knr = knr_rscv.predict(X_test)
# print('K Neighbors Regressor:')
# print('-' * 100)
# print('Train r2_score: ', r2_score(y_train, y_train_pred_knr))
# print('Test r2_score: ', r2_score(y_test, y_test_pred_knr))
# print('-' * 100)
# print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_knr)))
# print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_knr)))
# print('-' * 100)
# print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_knr) / y_train))
# print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test))
# error_knr = (np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test) / 2)

In [ ]:
# svr = SVR()
# possible_parameter_values = {'gamma' : [float(x)/10000 for x in range(100001)],
#                             'C' : [float(x)/10 for x in range(1001)]}
# svr_rscv = RandomizedSearchCV(estimator = svr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
#                               verbose = 2, n_jobs = 1)
# svr_rscv.fit(X_train, y_train)
# svr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [47]:
#y_train_pred_svr = svr_rscv.predict(X_train)
#y_test_pred_svr = svr_rscv.predict(X_test)
#print('Support Vector Regressor:')
#print('-' * 100)
#print('Train r2_score: ', r2_score(y_train, y_train_pred_svr))
#print('Test r2_score: ', r2_score(y_test, y_test_pred_svr))
#print('-' * 100)
#print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_svr)))
#print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_svr)))
#print('-' * 100)
#print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_svr) / y_train))
#print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_svr) / y_test))
#error_svr = np.mean(mean_absolute_error(y_test, y_test_pred_svr) / y_test)

In [24]:
xgbr = XGBRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)], 
                             'use_label_encoder' : [True, False], 
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)], 
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
xgbr_rscv = RandomizedSearchCV(estimator = xgbr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', 
                               verbose = 2, n_jobs = 1)
xgbr_rscv.fit(X_train, y_train)
xgbr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  22.9s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  20.7s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  20.4s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  20.9s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  21.3s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  21.3s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  21.1s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=  21.8s
[CV] END learning_rate=0.661, max_depth=80, n_estimators=250, use_label_encoder=True; total time=

[CV] END learning_rate=0.41, max_depth=75, n_estimators=100, use_label_encoder=False; total time=  46.8s
[CV] END learning_rate=0.41, max_depth=75, n_estimators=100, use_label_encoder=False; total time=  47.8s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  30.4s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  30.6s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  30.7s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  30.8s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  30.5s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  33.3s
[CV] END learning_rate=0.579, max_depth=50, n_estimators=350, use_label_encoder=True; total time=  31.2s
[CV] END learning_rate=0.579, max_depth=50, n_estimator

{'use_label_encoder': False,
 'n_estimators': 50,
 'max_depth': 80,
 'learning_rate': 0.094}

In [25]:
y_train_pred_xgbr = xgbr_rscv.predict(X_train)
y_test_pred_xgbr = xgbr_rscv.predict(X_test)
print('XGBoost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_xgbr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_xgbr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_xgbr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_xgbr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_xgbr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test))
error_xgbr = np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test)

XGBoost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.9885031290309186
Test r2_score:  0.28328685627324623
----------------------------------------------------------------------------------------------------
Train RMSE:  3.180318249364859
Test RMSE:  25.510683193267884
----------------------------------------------------------------------------------------------------
Train MAPE:  0.011488269776736426
Test MAPE:  0.11795967697528208


In [53]:
file = open(r'D:\\IPL\\ipl_score_prediction.pkl', 'wb')
pickle.dump(abr_rscv, file)
file.close()

In [59]:
'batting_team_Chennai_Super_Kings', 'batting_team_Delhi_Capitals',
'batting_team_Gujarat_Lions', 'batting_team_Kings_XI_Punjab',
'batting_team_Kolkata_Knight_Riders', 'batting_team_Mumbai_Indians',
'batting_team_Rajasthan_Royals', 'batting_team_Rising_Pune_Supergiant',
'batting_team_Royal_Challengers_Bangalore',
'batting_team_Sunrisers_Hyderabad', 'bowling_team_Chennai_Super_Kings',
'bowling_team_Delhi_Capitals', 'bowling_team_Gujarat_Lions',
'bowling_team_Kings_XI_Punjab', 'bowling_team_Kolkata_Knight_Riders',
'bowling_team_Mumbai_Indians', 'bowling_team_Rajasthan_Royals',
'bowling_team_Rising_Pune_Supergiant',
'bowling_team_Royal_Challengers_Bangalore',
'bowling_team_Sunrisers_Hyderabad', 'toss_decision_bat',
'toss_decision_field',
'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium',
'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens',
'venue_Feroz_Shah_Kotla', 'venue_Green_Park',
'venue_Holkar_Cricket_Stadium', 'venue_M_Chinnaswamy_Stadium',
'venue_MA_Chidambaram_Stadium_Chepauk',
'venue_Maharashtra_Cricket_Association_Stadium',
'venue_Punjab_Cricket_Association_IS_Bindra_Stadium_Mohali',
'venue_Rajiv_Gandhi_International_Stadium_Uppal',
'venue_Saurashtra_Cricket_Association_Stadium',
'venue_Sawai_Mansingh_Stadium',
'venue_Shaheed_Veer_Narayan_Singh_International_Stadium',
'venue_Sharjah_Cricket_Stadium', 'venue_Sheikh_Zayed_Stadium',
'venue_Wankhede_Stadium', 'inning', 'over', 'ball', 'runs', 'wickets',
'runs_last_5_overs', 'wickets_last_5_overs'

('runs_last_5_overs', 'wickets_last_5_overs')

In [60]:
arr = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 30, 1, 30, 1])

In [61]:
arr = arr.reshape(1, -1)

In [62]:
s = knr_rscv.predict(arr)[0]
s

165.5

In [63]:
s - (0.09185259348772307 * s)

150.29839577778182

In [64]:
s + (0.09185259348772307 * s)

180.70160422221818

In [31]:
int(5.3/1)

5

In [40]:
np.round((5.3%1)*10, 0)

3.0

In [35]:
np.round(7.4%1, 1)

0.4